 ## Import packages and modules

In [27]:
from datetime import date
from datetime import timedelta
import os
import datetime
import pandas as pd
import ee
import geemap
import os
import eeDatabase_coreMethods as eedb_cor
import eeDatabase_collectionMethods as eedb_col
import eeDatabase_collectionInfo as eedb_colinfo

# ee.Authenticate()
ee.Initialize(project = "dri-apps")

Map = geemap.Map()

## Define Parameters and Load datasets

In [32]:
# ------------------------------------- Define parameters -----------------------------------------------
# Define time period to export
start_date = datetime.datetime(1980, 1, 1)
end_date = datetime.datetime(2025, 1, 1)

# -------------------------------- Define input Image Collection ----------------------------------------
# Define input dataset
# See dictionary below for list of input datasets
in_ic_name = 'RAP_Production'

# Define variable from dataset
# See dictionary below for variables available for each dataset
var_name = 'SHR'

# ------------------------------- Define input Feature Collection ---------------------------------------
# Define input path for Feature Collection
in_fc_path = 'projects/dri-apps/assets/blm-admin/blm-natl-grazing-allotment-polygons'
in_fc = ee.FeatureCollection(in_fc_path)

# ------------------------------ Define mask, if applicable --------------------------------------------
# For BLM, we will apply mask to field offices, district offices, and state offices, but not to allotments
# Apply mask for ownership, landcover, or other variables. Must be binary mask.
mask_path = 'projects/dri-apps/assets/blm-admin/blm-natl-admu-sma-binary'

## Define additional parameters required for functions and output properties below

In [33]:
# Define land unit names
if(in_fc_path == 'projects/dri-apps/assets/blm-admin/blm-natl-grazing-allotment-polygons'):
    land_unit_long = 'BLM_Natl_Grazing_Allotment_Polygons'
    land_unit_short = 'BLM_Allotments'
    tile_scale = 1
    in_fc_id = 'ALLOT_ID'
    fc_mask = False
elif(in_fc_path == 'projects/dri-apps/assets/blm-admin/blm-natl-admu-fieldoffice-polygons'):
    land_unit_long = 'BLM_Natl_FieldOffice_Polygons'
    land_unit_short = 'BLM_FieldOffices'
    tile_scale = 1
    in_fc_id = 'FO_ID'
    fc_mask = True
elif(in_fc_path == 'projects/dri-apps/assets/blm-admin/blm-natl-admu-districtoffice-polygons'):
    land_unit_long = 'BLM_Natl_DistrictOffice_Polygons'
    land_unit_short = 'BLM_DistrictOffices'
    tile_scale = 1
    in_fc_id = 'DO_ID'
    fc_mask = True
elif(in_fc_path == 'projects/dri-apps/assets/blm-admin/blm-natl-admu-stateoffice-polygons'):
    land_unit_long = 'BLM_Natl_StateOffice_Polygons'
    land_unit_short = 'BLM_StateOffices'
    tile_scale = 1
    in_fc_id = 'SO_ID'
    fc_mask = True

# Pull out additional variables needed to run exports
in_ic_paths = eedb_colinfo.in_ic_dict.get(in_ic_name).get('in_ic_paths')
in_ic_res = ee.Number(ee.ImageCollection(in_ic_paths[0]).first().projection().nominalScale()).round().getInfo()
var_type = eedb_colinfo.in_ic_dict.get(in_ic_name).get('var_type')
var_units = eedb_colinfo.var_dict.get(var_name).get('units')
out_path = f"projects/climate-engine-pro/assets/blm-database/{land_unit_short.replace('_', '').lower()}-{in_ic_name.replace('_', '').lower()}-{var_name.replace('_', '').lower()}"

# Apply mask for remote sensing datasets on larger boundaries
if fc_mask == True & eedb_colinfo.in_ic_dict.get(in_ic_name).get('ic_mask') == True:
    mask = True
else:
    mask = False

## TEMPORARY

In [34]:
# Create dictionary of properties
properties = {'system:index': '0_id', 'land_unit_long': land_unit_long, 'land_unit_short': land_unit_short, 'in_fc_path': in_fc_path, 
              'in_fc_id': in_fc_id, 'in_ic_path': in_ic_paths[0], 'in_ic_name': in_ic_name, 'in_ic_res': in_ic_res, 'var_type': var_type, 
              'var_name': var_name, 'var_units': var_units, 'tile_scale': tile_scale}
if mask == True:
        properties['mask_path'] = mask_path
elif mask == False:
        properties['mask_path'] = 'None'

# Apply ID image function to input feature collection
out_list = eedb_cor.generate_id_img(in_fc_path = properties.get('in_fc_path'), in_fc_id = properties.get('in_fc_id'))
out_i = ee.Image(out_list.get(0))
out_fc = ee.FeatureCollection(out_list.get(1))

# Pull args out of properties for string parsing below
land_unit_short = properties.get('land_unit_short')
in_ic_name = properties.get('in_ic_name')
var_name = properties.get('var_name')

task = ee.batch.Export.image.toAsset(
        image = out_i.set(properties),
        description = f"initialize - {land_unit_short.replace('_', '').lower()} {in_ic_name.replace('_', '').lower()} {var_name.replace('_', '').lower()} - id",
        assetId = out_path + '/0_id',
        region = out_fc.geometry().buffer(20),
        scale = 22.264,
        maxPixels = 1e13)
task.start()

## Create database image collection and append time-series images

In [ ]:
# If there is no Image Collection asset at the out_path create one and export ID image
if os.system(f"earthengine asset info {out_path}") == 256:

    print("Initializing Image Collection by creating EE asset and exporting ID image")
    
    # Create dictionary of properties
    properties = {'system:index': '0_id', 'land_unit_long': land_unit_long, 'land_unit_short': land_unit_short, 'in_fc_path': in_fc_path, 
                  'in_fc_id': in_fc_id, 'in_ic_path': in_ic_paths[0], 'in_ic_name': in_ic_name, 'in_ic_res': in_ic_res, 'var_type': var_type, 
                  'var_name': var_name, 'var_units': var_units, 'tile_scale': tile_scale}
    if mask == True:
            properties['mask_path'] = mask_path
    elif mask == False:
            properties['mask_path'] = 'None'
    eedb_cor.initialize_collection(out_path = out_path, properties = properties)

    
# If there is an Image Collection asset at the out_path export time-series images
elif os.system(f"earthengine asset info {out_path}") == 0:

    print(f"Appending to Image Collection for dates {start_date} - {end_date}")

    # Get dates for image collection based on start and end date
    dates = eedb_cor.get_collection_dates(in_ic_paths = in_ic_paths, start_date = start_date, end_date = end_date)

    # Loop over selected dates to pre-process and export
    for date in dates:
        
        print("Running ", datetime.datetime.fromtimestamp(date/1000.0))
        
        # Parse date for ID
        date_ymd = datetime.datetime.fromtimestamp(date/1000.0).strftime('%Y%m%d')
        
        # Create dictionary of properties for image    
        properties = {'system:index': date_ymd, 'system:time_start': date, 'land_unit_long': land_unit_long, 'land_unit_short': land_unit_short, 'in_fc_path': in_fc_path,\
                      'in_fc_id': in_fc_id, 'in_ic_path': in_ic_paths[0], 'in_ic_name': in_ic_name, 'in_ic_res': in_ic_res, 'var_type': var_type,\
                      'var_name': var_name, 'var_units': var_units, 'tile_scale': tile_scale}
        if mask == True:
            properties['mask_path'] = mask_path
        elif mask == False:
            properties['mask_path'] = 'None'
        
        eedb_cor.run_image_export(in_ic_paths = in_ic_paths, date = date, out_path = out_path, properties = properties)

{
  "id": "projects/climate-engine-pro/assets/blm-database/blmstateoffices-rap16dproduction-shragb",
  "name": "projects/climate-engine-pro/assets/blm-database/blmstateoffices-rap16dproduction-shragb",
  "type": "IMAGE_COLLECTION",
  "updateTime": "2023-12-25T15:10:12.684249Z"
}
{
  "id": "projects/climate-engine-pro/assets/blm-database/blmstateoffices-rap16dproduction-shragb",
  "name": "projects/climate-engine-pro/assets/blm-database/blmstateoffices-rap16dproduction-shragb",
  "type": "IMAGE_COLLECTION",
  "updateTime": "2023-12-25T15:10:12.684249Z"
}
Appending to Image Collection for dates 1980-01-01 00:00:00 - 2025-01-01 00:00:00
Running  1985-12-31 17:00:00
Running  1986-01-16 17:00:00
Running  1986-02-01 17:00:00
Running  1986-02-17 17:00:00
Running  1986-03-05 17:00:00
Running  1986-03-21 17:00:00
Running  1986-04-06 17:00:00
Running  1986-04-22 17:00:00
Running  1986-05-08 17:00:00
Running  1986-05-24 17:00:00
Running  1986-06-09 17:00:00
Running  1986-06-25 17:00:00
Running  1

Running  1997-05-08 17:00:00
Running  1997-05-24 17:00:00
Running  1997-06-09 17:00:00
Running  1997-06-25 17:00:00
Running  1997-07-11 17:00:00
Running  1997-07-27 17:00:00
Running  1997-08-12 17:00:00
Running  1997-08-28 17:00:00
Running  1997-09-13 17:00:00
Running  1997-09-29 17:00:00
Running  1997-10-15 17:00:00
Running  1997-10-31 17:00:00
Running  1997-11-16 17:00:00
Running  1997-12-02 17:00:00
Running  1997-12-18 17:00:00
Running  1997-12-31 17:00:00
Running  1998-01-16 17:00:00
Running  1998-02-01 17:00:00
Running  1998-02-17 17:00:00
Running  1998-03-05 17:00:00
Running  1998-03-21 17:00:00
Running  1998-04-06 17:00:00
Running  1998-04-22 17:00:00
Running  1998-05-08 17:00:00
Running  1998-05-24 17:00:00
Running  1998-06-09 17:00:00
Running  1998-06-25 17:00:00
Running  1998-07-11 17:00:00
Running  1998-07-27 17:00:00
Running  1998-08-12 17:00:00
Running  1998-08-28 17:00:00
Running  1998-09-13 17:00:00
Running  1998-09-29 17:00:00
Running  1998-10-15 17:00:00
Running  1998-

Running  2009-08-28 17:00:00
Running  2009-09-13 17:00:00
Running  2009-09-29 17:00:00
Running  2009-10-15 17:00:00
Running  2009-10-31 17:00:00
Running  2009-11-16 17:00:00
Running  2009-12-02 17:00:00
Running  2009-12-18 17:00:00
Running  2009-12-31 17:00:00
Running  2010-01-16 17:00:00
Running  2010-02-01 17:00:00
Running  2010-02-17 17:00:00
Running  2010-03-05 17:00:00
Running  2010-03-21 17:00:00
Running  2010-04-06 17:00:00
Running  2010-04-22 17:00:00
Running  2010-05-08 17:00:00
Running  2010-05-24 17:00:00
Running  2010-06-09 17:00:00
Running  2010-06-25 17:00:00
Running  2010-07-11 17:00:00
Running  2010-07-27 17:00:00
Running  2010-08-12 17:00:00
Running  2010-08-28 17:00:00
Running  2010-09-13 17:00:00
Running  2010-09-29 17:00:00
Running  2010-10-15 17:00:00
Running  2010-10-31 17:00:00
Running  2010-11-16 17:00:00
Running  2010-12-02 17:00:00
Running  2010-12-18 17:00:00
Running  2010-12-31 17:00:00
Running  2011-01-16 17:00:00
Running  2011-02-01 17:00:00
Running  2011-

## Check completeness of image collections and re-run any images that are missing

In [25]:
# Get list of all dates
all_dates = eedb_cor.get_collection_dates(in_ic_paths, start_date, end_date)

# Get list of dates from collection
coll_dates = ee.ImageCollection(out_path).aggregate_array('system:time_start').distinct().getInfo()

# Get list of dates missing from collection
miss_dates = sorted(set(all_dates) - set(coll_dates))
print("These dates are missing and will be rerun ", miss_dates)

for date in miss_dates:
    print("Running ", datetime.datetime.fromtimestamp(date/1000.0))
    
    # Parse date for ID
    date_ymd = datetime.datetime.fromtimestamp(date/1000.0).strftime('%Y%m%d')

    # Create dictionary of properties for image    
    properties = {'system:index': date_ymd, 'system:time_start': date, 'land_unit_long': land_unit_long, 'land_unit_short': land_unit_short, 'in_fc_path': in_fc_path,\
                'in_fc_id': in_fc_id, 'in_ic_paths': in_ic_paths, 'in_ic_path': in_ic_paths[0], 'in_ic_name': in_ic_name, 'in_ic_res': in_ic_res, 'var_type': var_type,\
                'var_name': var_name, 'var_units': var_units, 'tile_scale': tile_scale, 'mask': mask}
    if mask == True:
        properties['mask_path'] = mask_path
    elif mask == False:
        properties['mask_path'] = 'None'
        
    eedb_cor.run_image_export(in_ic_paths = in_ic_paths, date = date, out_path = out_path, properties = properties)

These dates are missing and will be rerun  [1702706400000, 1703138400000]
Running  2023-12-15 23:00:00
Running  2023-12-20 23:00:00
